# 2. annotate MNV using VEP

In this section we will walk through how to annotate MNV using vep in hail.

In order to reproduce this tutorial results, you will need to set up your vep configuration. 
See [Hail docs](https://hail.is/docs/0.2/methods/genetics.html?highlight=vep#hail.methods.vep) for vep configuration set up.

Similar to the last tutorial, we ran

`cluster start mycluster --num-preemptible-workers 8 --vep --init gs://gnomad-public/tools/inits/master-init.sh`
Followed by
`cluster connect mycluster notebook`

To build the jupyter notebook in google cloud cluster.


In [1]:
#importing hail, and specifying the vep config
import hail as hl
import hail.expr.aggregators as agg
from typing import *
vep_config = "gs://gnomad-resources/loftee-beta/vep85-loftee-gcloud.json"  # this is the config that actually works!

In [3]:
#import the mnv table created in the tutorial step 1.
mnv = hl.read_table("gs://gnomad-qingbowang/MNV/demo_et_combined.ht")
mnv.show()

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.3
SparkUI available at http://10.128.0.44:4043
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.11-61a2083d5773
LOGGING: writing to /home/hail/hail-20190310-0052-0.2.11-61a2083d5773.log


+---------------+------------+---------------+--------------+-------+----------+-------+
| locus         | alleles    | prev_locus    | prev_alleles |  dist |       AF |    AC |
+---------------+------------+---------------+--------------+-------+----------+-------+
| locus<GRCh37> | array<str> | locus<GRCh37> | array<str>   | int32 |  float64 | int32 |
+---------------+------------+---------------+--------------+-------+----------+-------+
| 1:10506       | ["C","G"]  | 1:10505       | ["A","T"]    |     1 | 2.00e-04 |     1 |
| 1:13118       | ["A","G"]  | 1:13116       | ["T","G"]    |     2 | 9.70e-02 |   486 |
| 1:48328       | ["A","T"]  | 1:48327       | ["C","A"]    |     1 | 4.39e-03 |    22 |
| 1:49989       | ["A","G"]  | 1:49988       | ["T","A"]    |     1 | 5.79e-03 |    29 |
| 1:51049       | ["A","C"]  | 1:51047       | ["A","T"]    |     2 | 1.60e-03 |     8 |
| 1:51050       | ["A","T"]  | 1:51049       | ["A","C"]    |     1 | 1.60e-03 |     8 |
| 1:55165       | ["A

## annotating the SNP effects

This is relatively straightforward. Let the SNP of `prev_locus` be SNP1, and the other one be SNP2 (i.e. SNP2 is downstream).

Since the vep function in hail takes `locus` and `alleles`, it is easier if we first annotate the SNP2 consequence by vep:

In [7]:
mnv = mnv.key_by("locus","alleles")
mnv = hl.vep(mnv, vep_config, name="snp2_vep")

Next we want to annotate the SNP1 consequence using vep. In order to do this, we need to change the row name a little bit:

In [10]:
mnv = mnv.key_by() #unkey first
mnv = mnv.rename({'locus' : 'snp2_locus', 'alleles' : 'snp2_alleles',"prev_locus":"locus", "prev_alleles":"alleles"}) #rename the snp1 locus and alleles as locus, allele
mnv = mnv.key_by('locus', 'alleles') #and re-key
mnv = hl.vep(mnv, vep_config, name="snp1_vep") #and vep


2019-03-10 01:00:19 Hail: INFO: Coerced sorted dataset


Now let's see that the individual effects are correctly annotated by vep:

In [11]:
mnv.show(5) #check

2019-03-10 01:01:03 Hail: INFO: Coerced sorted dataset


+---------------+--------------+---------------+------------+-------+----------+-------+
| snp2_locus    | snp2_alleles | locus         | alleles    |  dist |       AF |    AC |
+---------------+--------------+---------------+------------+-------+----------+-------+
| locus<GRCh37> | array<str>   | locus<GRCh37> | array<str> | int32 |  float64 | int32 |
+---------------+--------------+---------------+------------+-------+----------+-------+
| 1:10506       | ["C","G"]    | 1:10505       | ["A","T"]  |     1 | 2.00e-04 |     1 |
| 1:13118       | ["A","G"]    | 1:13116       | ["T","G"]  |     2 | 9.70e-02 |   486 |
| 1:48328       | ["A","T"]    | 1:48327       | ["C","A"]  |     1 | 4.39e-03 |    22 |
| 1:49989       | ["A","G"]    | 1:49988       | ["T","A"]  |     1 | 5.79e-03 |    29 |
| 1:51049       | ["A","C"]    | 1:51047       | ["A","T"]  |     2 | 1.60e-03 |     8 |
+---------------+--------------+---------------+------------+-------+----------+-------+

+----------+--------

## annotating the MNV effects

This is still relatively straightforward, when the SNPs are next to each other, so let's focus on such MNVs first and do the annotation:

In [16]:
#let t1 be those with distance==1
t1 = mnv.filter(mnv.dist==1)
t1 = t1.annotate(refs=t1.alleles[0] +t1.snp2_alleles[0], alts=t1.alleles[1] +t1.snp2_alleles[1])# annotate combined refs, and alts
t1 = t1.annotate(mnv_alleles = [t1.refs,t1.alts]) #and let it be mnv_alleles
t1 = t1.key_by()#to re-key
t1 = t1.rename({'alleles' : 'snp1_alleles',"mnv_alleles":"alleles"}) #now the alleles is that of mnv (note. locus does not change. same as snp1)
t1 = t1.key_by('locus', 'alleles') #and re-key
t1 = hl.vep(t1, vep_config, name="mnv_vep")

2019-03-10 01:07:34 Hail: INFO: Coerced sorted dataset
2019-03-10 01:07:34 Hail: INFO: Coerced sorted dataset


In [17]:
t1.show(5) #check

2019-03-10 01:07:44 Hail: INFO: Coerced sorted dataset
2019-03-10 01:07:49 Hail: INFO: Coerced sorted dataset


+---------------+--------------+---------------+--------------+-------+----------+-------+
| snp2_locus    | snp2_alleles | locus         | snp1_alleles |  dist |       AF |    AC |
+---------------+--------------+---------------+--------------+-------+----------+-------+
| locus<GRCh37> | array<str>   | locus<GRCh37> | array<str>   | int32 |  float64 | int32 |
+---------------+--------------+---------------+--------------+-------+----------+-------+
| 1:10506       | ["C","G"]    | 1:10505       | ["A","T"]    |     1 | 2.00e-04 |     1 |
| 1:48328       | ["A","T"]    | 1:48327       | ["C","A"]    |     1 | 4.39e-03 |    22 |
| 1:49989       | ["A","G"]    | 1:49988       | ["T","A"]    |     1 | 5.79e-03 |    29 |
| 1:51050       | ["A","T"]    | 1:51049       | ["A","C"]    |     1 | 1.60e-03 |     8 |
| 1:55165       | ["A","G"]    | 1:55164       | ["C","A"]    |     1 | 2.00e-04 |     1 |
+---------------+--------------+---------------+--------------+-------+----------+-------+

When the SNPs are not to each other, it is a bit not straightforward because you need to fill the reference base pair that are in between, 
in order to pass to vep. 
Fortunately, our nice hail also allows us to do that. The first step is to load the reference genome:

In [18]:
grch37 = hl.get_reference('GRCh37')
grch37_fasta = 'gs://hail-common/references/human_g1k_v37.fasta.gz'
grch37_fai = 'gs://hail-common/references/human_g1k_v37.fasta.fai'
grch37.add_sequence(grch37_fasta, grch37_fai)


Then we can fill the context using `sequence_context`:

In [48]:
t2 = mnv.filter(mnv.dist==2)
t2 = t2.annotate(refs = t2.alleles[0] + t2.locus.sequence_context(before=-1, after=1) + t2.snp2_alleles[0],
                           alts = t2.alleles[1] + t2.locus.sequence_context(before=-1, after=1) + t2.snp2_alleles[1]) #fill the one reference sequence after the snp1

In [49]:
t2.show()

2019-03-10 02:01:48 Hail: INFO: Coerced sorted dataset


+---------------+--------------+---------------+------------+-------+----------+-------+
| snp2_locus    | snp2_alleles | locus         | alleles    |  dist |       AF |    AC |
+---------------+--------------+---------------+------------+-------+----------+-------+
| locus<GRCh37> | array<str>   | locus<GRCh37> | array<str> | int32 |  float64 | int32 |
+---------------+--------------+---------------+------------+-------+----------+-------+
| 1:13118       | ["A","G"]    | 1:13116       | ["T","G"]  |     2 | 9.70e-02 |   486 |
| 1:51049       | ["A","C"]    | 1:51047       | ["A","T"]  |     2 | 1.60e-03 |     8 |
| 1:57264       | ["T","G"]    | 1:57262       | ["T","G"]  |     2 | 1.40e-03 |     7 |
| 1:74792       | ["G","A"]    | 1:74790       | ["C","G"]  |     2 | 3.77e-02 |   189 |
| 1:362907      | ["G","C"]    | 1:362905      | ["T","G"]  |     2 | 2.18e-02 |   109 |
| 1:559985      | ["C","T"]    | 1:559983      | ["C","T"]  |     2 | 1.40e-02 |    70 |
| 1:564862      | ["T

And we are good to go!

In [50]:
t2 = t2.annotate(mnv_alleles = [t2.refs,t2.alts]) #and let it be mnv_alleles
t2 = t2.key_by()#to re-key
t2 = t2.rename({'alleles' : 'snp1_alleles',"mnv_alleles":"alleles"}) #now the alleles is that of mnv (note. locus does not change. same as snp1)
t2 = t2.key_by('locus', 'alleles') #and re-key
t2 = hl.vep(t2, vep_config, name="mnv_vep", block_size=10)

2019-03-10 02:02:48 Hail: INFO: Coerced sorted dataset
2019-03-10 02:02:48 Hail: INFO: Coerced sorted dataset


In [51]:
t2.show()

2019-03-10 02:02:57 Hail: INFO: Coerced sorted dataset
2019-03-10 02:02:58 Hail: INFO: Coerced sorted dataset


+---------------+--------------+---------------+--------------+-------+----------+-------+
| snp2_locus    | snp2_alleles | locus         | snp1_alleles |  dist |       AF |    AC |
+---------------+--------------+---------------+--------------+-------+----------+-------+
| locus<GRCh37> | array<str>   | locus<GRCh37> | array<str>   | int32 |  float64 | int32 |
+---------------+--------------+---------------+--------------+-------+----------+-------+
| 1:13118       | ["A","G"]    | 1:13116       | ["T","G"]    |     2 | 9.70e-02 |   486 |
| 1:51049       | ["A","C"]    | 1:51047       | ["A","T"]    |     2 | 1.60e-03 |     8 |
| 1:57264       | ["T","G"]    | 1:57262       | ["T","G"]    |     2 | 1.40e-03 |     7 |
| 1:74792       | ["G","A"]    | 1:74790       | ["C","G"]    |     2 | 3.77e-02 |   189 |
| 1:362907      | ["G","C"]    | 1:362905      | ["T","G"]    |     2 | 2.18e-02 |   109 |
| 1:559985      | ["C","T"]    | 1:559983      | ["C","T"]    |     2 | 1.40e-02 |    70 |

## Comparing the effects and categorizing MNV

Now we have individual effect of SNP1, SNP2, as well as combined effect of MNV annotated. 
Next step is to compare those to categorize the MNVs. 

Are any "gained" nonsense in our dataset? How many MNVs are actually within a single codon reading frame?

To answer these question, we will first clean the data and get per transcript annotation, focusing on canonical transcripts. 

(We will start from those with distance=1. Same logic can be applied to all the processes for distance 2 later.)

(From here on, we are using some home made functions that are helpful for the analysis. 
Although there should be a more efficienc way to import those functions, here for the convenience we will copy and paste in the bottom of this notebook. 
See the "functions" cell for the functions that are used. In practice we need to define those functions first before running this process.)

In [34]:
#first, select the essential columns (delete some)
vepped_d1_essense = t1.key_by("locus", "refs", "alts") 
vepped_d1_essense = vepped_d1_essense.select("AC", "prev_AC", "prev_AF", "n_hethet", "n_hethom",
                                                     "n_homhom", "snp1_vep", "snp2_vep", "mnv_vep")
# filter to canonicals:
canon_cons_d1 = filter_vep_to_canonical_transcripts(filter_vep_to_canonical_transcripts(
filter_vep_to_canonical_transcripts(vepped_d1_essense, vep_root="snp1_vep"), vep_root="snp2_vep"),
                                                            vep_root="mnv_vep")

###explode by consequence, since there might be more than 1 canonical transcript for a MNV
canon_cons_d1 = canon_cons_d1.annotate(indices=hl.range(0, hl.min(
            hl.len(canon_cons_d1.snp1_vep.transcript_consequences), hl.len(canon_cons_d1.snp2_vep.transcript_consequences))))
canon_cons_d1 = canon_cons_d1.explode('indices')
canon_cons_d1 = canon_cons_d1.transmute(
            snp1_transcript_consequences=canon_cons_d1.snp1_vep.transcript_consequences[canon_cons_d1.indices],
            snp2_transcript_consequences=canon_cons_d1.snp2_vep.transcript_consequences[canon_cons_d1.indices],
            mnv_transcript_consequences=canon_cons_d1.mnv_vep.transcript_consequences[canon_cons_d1.indices],
            )
# keep necessary columns
canon_cons_d1 = canon_cons_d1.select("AC", "prev_AC", "prev_AF", "n_hethet", "n_hethom",
                                             "n_homhom",
                                             "snp1_transcript_consequences",
                                             "snp2_transcript_consequences",
                                             "mnv_transcript_consequences")

# further annotate necessary ones
# codons, consequence_terms, lof, lof_flags, transcript_id
canon_cons_d1 = canon_cons_d1.annotate(
            snp1_cons_term=canon_cons_d1.snp1_transcript_consequences.consequence_terms,
            snp2_cons_term=canon_cons_d1.snp2_transcript_consequences.consequence_terms,
            mnv_cons_term=canon_cons_d1.mnv_transcript_consequences.consequence_terms,
            snp1_codons=canon_cons_d1.snp1_transcript_consequences.codons,
            snp2_codons=canon_cons_d1.snp2_transcript_consequences.codons,
            mnv_codons=canon_cons_d1.mnv_transcript_consequences.codons,
            snp1_amino_acids=canon_cons_d1.snp1_transcript_consequences.amino_acids,
            snp2_amino_acids=canon_cons_d1.snp2_transcript_consequences.amino_acids,
            mnv_amino_acids=canon_cons_d1.mnv_transcript_consequences.amino_acids,
            snp1_lof=canon_cons_d1.snp1_transcript_consequences.lof,
            snp2_lof=canon_cons_d1.snp2_transcript_consequences.lof,
            mnv_lof=canon_cons_d1.mnv_transcript_consequences.lof,
            transcript_id=canon_cons_d1.snp1_transcript_consequences.transcript_id
            )

# filter to those that the codon are changed within a single reading frame
canon_cons_d1 = canon_cons_d1.filter(
            (canon_cons_d1.snp1_codons.length() == 7) & (canon_cons_d1.snp2_codons.length() == 7) & (
            canon_cons_d1.mnv_codons.length() == 7))

In above we have filtered to select MNVs only falling on canonical transcripts, within a single codon reading frame.


In [35]:
canon_cons_d1.show()

2019-03-10 01:47:58 Hail: INFO: Coerced sorted dataset
2019-03-10 01:48:03 Hail: INFO: Coerced sorted dataset
2019-03-10 01:48:08 Hail: INFO: Coerced sorted dataset


+---------------+------+------+-------+---------+----------+----------+----------+
| locus         | refs | alts |    AC | prev_AC |  prev_AF | n_hethet | n_hethom |
+---------------+------+------+-------+---------+----------+----------+----------+
| locus<GRCh37> | str  | str  | int32 |   int32 |  float64 |    int64 |    int64 |
+---------------+------+------+-------+---------+----------+----------+----------+
| 1:901922      | "GC" | "AA" |    53 |      53 | 1.06e-02 |       51 |        1 |
| 1:909238      | "GT" | "CC" |     1 |    3889 | 7.77e-01 |        0 |        0 |
+---------------+------+------+-------+---------+----------+----------+----------+

+----------+-----------------------------------------+
| n_homhom | snp1_transcript_consequences.allele_num |
+----------+-----------------------------------------+
|    int64 |                                   int32 |
+----------+-----------------------------------------+
|        0 |                                       1 |
|    

For the last step, we will actually use traditional dataframe approach, by turning the table to pandas dataframe.
(the flexibility of `apply` command pandas has is still attractive!)

Then we can annotate the most severe consequence out of all the possible consequences, 
and also categorize the MNV by comparing the consequence of SNP1, SNP2, and MNV.

In [37]:
#keep the necessary columns and turn to pandas dataframe
canon_cons_pd1 = canon_cons_d1.select("snp1_cons_term", "snp2_cons_term", "mnv_cons_term", "snp1_codons",
                                              "snp2_codons", "mnv_codons", "snp1_amino_acids", "snp2_amino_acids",
                                              "mnv_amino_acids", "snp1_lof", "snp2_lof", "mnv_lof", "transcript_id",
                                              "AC", "prev_AC", "n_hethet", "n_hethom", "n_homhom").to_pandas()
# get the most severe consequence
canon_cons_pd1["snp1_sev"] = canon_cons_pd1.snp1_cons_term.apply(lambda x: cons_term_most_severe(x))
canon_cons_pd1["snp2_sev"] = canon_cons_pd1.snp2_cons_term.apply(lambda x: cons_term_most_severe(x))
canon_cons_pd1["mnv_sev"] = canon_cons_pd1.mnv_cons_term.apply(lambda x: cons_term_most_severe(x))
# annotate the category
canon_cons_pd1["categ"] = canon_cons_pd1.apply(
            lambda x: mnv_category(x["snp1_sev"], x["snp2_sev"], x["mnv_sev"], x["snp1_amino_acids"],
                                   x["snp2_amino_acids"],
                                   x["mnv_amino_acids"]), axis=1)
#rewrite lof flags just in case if lof column is all None (NA) and that causes error
canon_cons_pd1.snp1_lof = canon_cons_pd1.snp1_lof.astype(str)
canon_cons_pd1.snp2_lof = canon_cons_pd1.snp2_lof.astype(str)
canon_cons_pd1.mnv_lof = canon_cons_pd1.mnv_lof.astype(str)


2019-03-10 01:50:09 Hail: INFO: Coerced sorted dataset
2019-03-10 01:50:09 Hail: INFO: Coerced sorted dataset
2019-03-10 01:50:10 Hail: INFO: Coerced sorted dataset


In [38]:
display(canon_cons_pd1)

,locus.contig,locus.position,refs,alts,snp1_cons_term,snp2_cons_term,mnv_cons_term,snp1_codons,snp2_codons,mnv_codons,...,transcript_id,AC,prev_AC,n_hethet,n_hethom,n_homhom,snp1_sev,snp2_sev,mnv_sev,categ
0,1,901922,GC,AA,[missense_variant],[missense_variant],[missense_variant],aGc/aAc,agC/agA,aGC/aAA,...,ENST00000379410,53,53,51,1,0,missense_variant,missense_variant,missense_variant,Changed missense
1,1,909238,GT,CC,[missense_variant],[synonymous_variant],[missense_variant],cGt/cCt,cgT/cgC,cGT/cCC,...,ENST00000379410,1,3889,0,0,1,missense_variant,synonymous_variant,missense_variant,Unchanged


## End notes

- After all, it turns out that there are only 2 MNVs of distance 1 that falls within a codon reading frame. One of them is "Changed missense", and the other is "Unchanged".

(Not surprising considering that we were just playing around with a small chunk of region -- again the very useful case of cloud computation is when dealing with larger dataset.)

- We can do pretty much the same thing in MNVs of distance 2 (just replace the `vepped_d1_essense = t1.key_by("locus", "refs", "alts") ` with 
`vepped_d2_essense = t2.key_by("locus", "refs", "alts") ` and all the downstream `d1` to `d2` for clarification.).

- (also note that running vep after annotating the sequence context sometimes raise, because of memory issue. In such case we recommend to increase the number of clusters.)

- Again we do provide a function `annotate_vep_mnv` that does all the processes above in a single line. Note that we need to specify the distance of the MNV for this function (i.e. you will run this function 2 times, one for distance 1, and another for distance 2)

- In the next section `functional_impact.ipynb`, we will use the real summary data from `gnomAD` exome -- a collection of 31,510 MNVs falling within a single codon!


## Functions
Below are the list of functions we defined in this notebook (before running all the processes above).

In [33]:
def filter_vep_to_canonical_transcripts(mt: Union[hl.MatrixTable, hl.Table],
                                        vep_root: str = 'vep') -> Union[hl.MatrixTable, hl.Table]:
    canonical = mt[vep_root].transcript_consequences.filter(lambda csq: csq.canonical == 1)
    vep_data = mt[vep_root].annotate(transcript_consequences=canonical)
    return mt.annotate_rows(**{vep_root: vep_data}) if isinstance(mt, hl.MatrixTable) else mt.annotate(**{vep_root: vep_data})

def mnv_category_by_aa_change(snp1_con, snp2_con, mnv_con,aa1,aa2,aa3):
    if (snp1_con, snp2_con, mnv_con)==("synonymous_variant","missense_variant","missense_variant"):
        if aa2==aa3: return ("Unchanged")
        else: return ("Changed missense")
    elif (snp1_con, snp2_con, mnv_con)==("missense_variant","synonymous_variant","missense_variant"):
        if aa1==aa3: return ("Unchanged")
        else: return ("Changed missense")
    elif (snp1_con, snp2_con, mnv_con)==("missense_variant","missense_variant","missense_variant"):
        if ((aa1==aa3) or (aa2==aa3)):
            return ("Partially changed missense")
        else: return ("Changed missense")
    else: return ("something wrong going on")

def cons_term_most_severe(cons_term_array):
    if "start_lost" in cons_term_array: return ("start_lost") #by definition this is sufficient to determine the mnv is uninteresting
    elif "stop_lost" in cons_term_array: return ("stop_lost")
    elif "stop_gained" in cons_term_array: return ("stop_gained")
    elif "missense_variant" in cons_term_array: return ("missense_variant")
    elif "stop_retained_variant" in cons_term_array: return ("stop_retained_variant")
    elif "synonymous_variant" in cons_term_array: return ("synonymous_variant")
    else: return ("Noncoding_or_else")

def mnv_category(snp1_con, snp2_con, mnv_con, aa1, aa2, aa3):
    # return the MNV consequence category, such as gained PTV, unchange, etc.
    # just classify everything of 3*3*3=27 pattern.
    #plus, case where we see start_lost / stop_lost
    # and if undeterminisitic, look at the aa change and determine.
    if snp1_con == "synonymous_variant":
        if snp2_con == "synonymous_variant":
            if mnv_con == "synonymous_variant":
                return ("Unchanged")
            elif mnv_con == "missense_variant":
                return ("Gained missense")
            elif mnv_con == "stop_gained":
                return ("Gained PTV")
            else:
                return ("Noncoding_or_else")
        if snp2_con == "missense_variant":
            if mnv_con == "synonymous_variant":
                return ("Lost missense")
            elif mnv_con == "missense_variant":
                return (mnv_category_by_aa_change(snp1_con, snp2_con, mnv_con, aa1, aa2, aa3))  # go look at aa change.
            elif mnv_con == "stop_gained":
                return ("Gained PTV")
            else:
                return ("Noncoding_or_else")
        if snp2_con == "stop_gained":
            if mnv_con == "synonymous_variant":
                return ("Rescued PTV")
            elif mnv_con == "missense_variant":
                return ("Rescued PTV")
            elif mnv_con == "stop_gained":
                return ("Unchanged")
            else:
                return ("Noncoding_or_else")
    if snp1_con == "missense_variant":
        if snp2_con == "synonymous_variant":
            if mnv_con == "synonymous_variant":
                return ("Lost missense")
            elif mnv_con == "missense_variant":
                return (mnv_category_by_aa_change(snp1_con, snp2_con, mnv_con, aa1, aa2, aa3))  # go look at aa change.
            elif mnv_con == "stop_gained":
                return ("Gained PTV")
            else:
                return ("Noncoding_or_else")
        if snp2_con == "missense_variant":
            if mnv_con == "synonymous_variant":
                return ("Lost missense")
            elif mnv_con == "missense_variant":
                return (mnv_category_by_aa_change(snp1_con, snp2_con, mnv_con, aa1, aa2, aa3))  # go look at aa change.
            elif mnv_con == "stop_gained":
                return ("Gained PTV")
            else:
                return ("Noncoding_or_else")
        if snp2_con == "stop_gained":
            if mnv_con == "synonymous_variant":
                return ("Rescued PTV")
            elif mnv_con == "missense_variant":
                return ("Rescued PTV")
            elif mnv_con == "stop_gained":
                return ("Unchanged")
            else:
                return ("Noncoding_or_else")
        else:
            return ("Noncoding_or_else")
    if snp1_con == "stop_gained":
        if snp2_con == "synonymous_variant":
            if mnv_con == "synonymous_variant":
                return ("Rescued PTV")
            elif mnv_con == "missense_variant":
                return ("Rescued PTV")
            elif mnv_con == "stop_gained":
                return ("Unchanged")
            else:
                return ("Noncoding_or_else")
        if snp2_con == "missense_variant":
            if mnv_con == "synonymous_variant":
                return ("Rescued PTV")
            elif mnv_con == "missense_variant":
                return ("Rescued PTV")
            elif mnv_con == "stop_gained":
                return ("Unchanged")
            else:
                return ("Noncoding_or_else")
        if snp2_con == "stop_gained":
            if mnv_con == "synonymous_variant":
                return ("Rescued PTV")
            elif mnv_con == "missense_variant":
                return ("Rescued PTV")
            elif mnv_con == "stop_gained":
                return ("Unchanged")
            else:
                return ("Noncoding_or_else")
        else:
            return ("Noncoding_or_else")
    #else, involving start_loss etc -> look at mnv cons first.
    elif mnv_con=="start_lost": return "Unchanged" #by definition individual effect is also start loss
    elif mnv_con=="stop_lost":
        if ((snp1_con=="stop_retained_variant") & (snp2_con=="stop_retained_variant")): return "gained_stop_loss"
        else: return ("Unchanged")
    elif mnv_con=="stop_retained_variant":#this case, by definition one of the variant is stop_lost, and the other is stop_retained
        return ("Rescued stop loss")
    else:
        return ("Noncoding_or_else")
